In [3]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_9593/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [5]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2_sup")
        # self.discard_below_average(sort_by="mse_sup")
        # self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results_05.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_25_9_3,0.954619,0.855043,0.975790,0.942036,0.947947,0.064411,0.205745,0.025337,0.288771,0.157054,0.353345,0.253794,1.001948,0.264598,1171.484934,1882.089540,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
1,model_25_9_4,0.954668,0.855042,0.975999,0.941725,0.947726,0.064342,0.205747,0.025118,0.290321,0.157720,0.353195,0.253658,1.001946,0.264456,1171.487079,1882.091685,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
2,model_25_9_2,0.954555,0.855027,0.975564,0.942330,0.948150,0.064503,0.205768,0.025574,0.287308,0.156441,0.353496,0.253975,1.001951,0.264787,1171.482083,1882.086689,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
3,model_25_9_5,0.954701,0.855025,0.976191,0.941397,0.947488,0.064295,0.205772,0.024917,0.291956,0.158437,0.353043,0.253565,1.001945,0.264360,1171.488541,1882.093147,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
4,model_25_9_1,0.954473,0.854993,0.975320,0.942605,0.948334,0.064619,0.205816,0.025829,0.285938,0.155883,0.353647,0.254202,1.001955,0.265024,1171.478508,1882.083114,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,model_35_1_24,0.951588,0.824005,0.869332,-3.174650,0.735884,0.068714,0.249800,0.122958,0.147519,0.135238,0.914305,0.262133,1.001433,0.273292,1675.355615,2693.116928,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
1031,model_34_2_16,0.937979,0.824004,0.472229,0.648232,0.626913,0.088030,0.249800,0.362215,0.180477,0.271346,0.328203,0.296699,1.001835,0.309330,1674.860155,2692.621468,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
1032,model_32_6_0,0.996647,0.823969,0.945046,0.986520,0.988294,0.004760,0.249850,0.014336,0.013580,0.013958,0.092494,0.068990,1.000144,0.071927,1176.695162,1887.299768,"Hidden Size=[26, 18], regularizer=0.5, learnin..."
1033,model_41_8_0,0.943423,0.823944,0.835315,0.628882,0.841478,0.080302,0.249886,0.158530,0.039617,0.099073,1.070168,0.283377,1.004297,0.295441,685.043910,1099.461691,"Hidden Size=[21, 12], regularizer=0.2, learnin..."
